## **Doenças do coração**
### REDES NEURAIS ARTIFICIAIS: CLASSIFICAÇÃO

**Legenda das colunas**

Age = idade (anos)

Sex = sexo (0=M; 1=F)

Chest Pain Type = tipo de dor no peito (0=TA:angina típica; 1=ATA:angina atípica; 2=NAP:dor não anginosa; 3=ASY:assintomático)

Resting BP = pressão sanguínea em repouso (mmHg)

Cholesterol = colesterol sérico (mg/dl)

Fasting BS = açúcar no sangue em jejum (mg/dl)
0: Fasting BS < 120 mg/dl (não diabético)
1: Fasting BS >= 120 mg/dl, (diabético)

Resting ECG = eletrocardiograma em repouso (0=Normal; 1=ST:Anormalidade da onda ST-T; 2=LVH: Hipertrofia ventricular esquerda)

Max HR = frequência cardíaca máxima

Exercise Angina = Angina induzida por exercício (0=Não; 1=Sim)

Old Peak = Depressão de ST induzida por exercício em relação ao repouso

ST_Slope = Inclinação do segmento ST (0=UP; 1=Flat; 2=Down)

Heart Disease = Doença cardíaca (0=Não possui doença cardíaca; 1=Possui doença cardíaca)

### Importação das bibliotecas

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

warnings.filterwarnings('ignore')
plt.style.use('dark_background')

df = pd.read_csv('/Users/diogomonteiro/heart_tratado.csv', sep=';', encoding = 'iso-8859-1')
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289.0,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180.0,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283.0,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214.0,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195.0,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
912,45,M,TA,110,264.0,0,Normal,132,N,1.2,Flat,1
913,68,M,ASY,144,193.0,1,Normal,141,N,3.4,Flat,1
914,57,M,ASY,130,131.0,0,Normal,115,Y,1.2,Flat,1
915,57,F,ATA,130,236.0,0,LVH,174,N,0.0,Flat,1


### Transformando as classes strings em variáveis categóricas ordinais

In [39]:
df1 = pd.get_dummies(df, columns = ['ChestPainType','RestingECG','ST_Slope'])
df1[['ChestPainType_ASY','ChestPainType_ATA', 'ChestPainType_NAP', 'ChestPainType_TA',
       'RestingECG_LVH', 'RestingECG_Normal', 'RestingECG_ST', 'ST_Slope_Down',
       'ST_Slope_Flat', 'ST_Slope_Up']] = df1[['ChestPainType_ASY','ChestPainType_ATA', 'ChestPainType_NAP', 'ChestPainType_TA',
       'RestingECG_LVH', 'RestingECG_Normal', 'RestingECG_ST', 'ST_Slope_Down',
       'ST_Slope_Flat', 'ST_Slope_Up']].astype('int64')

In [40]:
df1['Sex'].replace({'M': 0, 'F': 1}, inplace=True)
df1['ExerciseAngina'].replace({'N': 0, 'Y': 1}, inplace=True)
df1

,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,HeartDisease,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,40,0,140,289.0,0,172,0,0.0,0,0,1,0,0,0,1,0,0,0,1
1,49,1,160,180.0,0,156,0,1.0,1,0,0,1,0,0,1,0,0,1,0
2,37,0,130,283.0,0,98,0,0.0,0,0,1,0,0,0,0,1,0,0,1
3,48,1,138,214.0,0,108,1,1.5,1,1,0,0,0,0,1,0,0,1,0
4,54,0,150,195.0,0,122,0,0.0,0,0,0,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,45,0,110,264.0,0,132,0,1.2,1,0,0,0,1,0,1,0,0,1,0
913,68,0,144,193.0,1,141,0,3.4,1,1,0,0,0,0,1,0,0,1,0
914,57,0,130,131.0,0,115,1,1.2,1,1,0,0,0,0,1,0,0,1,0
915,57,1,130,236.0,0,174,0,0.0,1,0,1,0,0,1,0,0,0,1,0


### Atributos previsores e alvo

In [42]:
x = df1.drop('HeartDisease',axis=1)
previsores = x.iloc[:,0:19].values
previsores

array([[ 40.,   0., 140., ...,   0.,   0.,   1.],
       [ 49.,   1., 160., ...,   0.,   1.,   0.],
       [ 37.,   0., 130., ...,   0.,   0.,   1.],
       ...,
       [ 57.,   0., 130., ...,   0.,   1.,   0.],
       [ 57.,   1., 130., ...,   0.,   1.,   0.],
       [ 38.,   0., 138., ...,   0.,   0.,   1.]])

In [43]:
previsores.shape

(917, 18)

In [44]:
alvo = df1.iloc[:,8].values
alvo

array([0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,

In [45]:
alvo.shape

(917,)

### **Análise das escalas dos atributos (Escalonamento) e pré-processamento**

Padronização (Ultiliza a média e o desvio padrão como referência).

Normalização (Ultiliza os valores máximo e mínimo como referência).

In [48]:
x.describe()

,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
count,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000
mean,53.509269,0.210469,132.540894,244.635389,0.233370,136.789531,0.404580,0.886696,0.540894,0.188659,0.220284,0.050164,0.205016,0.600872,0.194111,0.068702,0.500545,0.430752
std,9.437636,0.407864,17.999749,53.347125,0.423206,25.467129,0.491078,1.066960,0.498597,0.391451,0.414664,0.218401,0.403934,0.489986,0.395731,0.253085,0.500273,0.495452
min,28.000000,0.000000,80.000000,85.000000,0.000000,60.000000,0.000000,-2.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.000000,0.000000,120.000000,214.000000,0.000000,120.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,54.000000,0.000000,130.000000,244.635389,0.000000,138.000000,0.000000,0.600000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,60.000000,0.000000,140.000000,267.000000,0.000000,156.000000,1.000000,1.500000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,77.000000,1.000000,200.000000,603.000000,1.000000,202.000000,1.000000,6.200000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [49]:
previsores_esc = StandardScaler().fit_transform(previsores)

In [50]:
previsores_esc

array([[-1.43220634, -0.51630861,  0.41462669, ..., -0.27160724,
        -1.00109111,  1.14957339],
       [-0.47805725,  1.9368261 ,  1.52635965, ..., -0.27160724,
         0.99891008, -0.86988791],
       [-1.75025603, -0.51630861, -0.14123979, ..., -0.27160724,
        -1.00109111,  1.14957339],
       ...,
       [ 0.37007527, -0.51630861, -0.14123979, ..., -0.27160724,
         0.99891008, -0.86988791],
       [ 0.37007527,  1.9368261 , -0.14123979, ..., -0.27160724,
         0.99891008, -0.86988791],
       [-1.64423947, -0.51630861,  0.30345339, ..., -0.27160724,
        -1.00109111,  1.14957339]])

In [51]:
previsoresdf = pd.DataFrame(previsores_esc)
previsoresdf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-1.432206,-0.516309,0.414627,0.832075,-0.551733,1.383339,-0.824310,-0.831502,-1.085425,2.073784,-0.531524,-0.229810,-0.507826,0.815013,-0.490781,-0.271607,-1.001091,1.149573
1,-0.478057,1.936826,1.526360,-1.212261,-0.551733,0.754736,-0.824310,0.106251,-1.085425,-0.482210,1.881384,-0.229810,-0.507826,0.815013,-0.490781,-0.271607,0.998910,-0.869888
2,-1.750256,-0.516309,-0.141240,0.719543,-0.551733,-1.523953,-0.824310,-0.831502,-1.085425,2.073784,-0.531524,-0.229810,-0.507826,-1.226974,2.037569,-0.271607,-1.001091,1.149573
3,-0.584074,1.936826,0.303453,-0.574578,-0.551733,-1.131075,1.213136,0.575128,0.921298,-0.482210,-0.531524,-0.229810,-0.507826,0.815013,-0.490781,-0.271607,0.998910,-0.869888
4,0.052026,-0.516309,0.970493,-0.930931,-0.551733,-0.581047,-0.824310,-0.831502,-1.085425,-0.482210,1.881384,-0.229810,-0.507826,0.815013,-0.490781,-0.271607,-1.001091,1.149573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,-0.902124,-0.516309,-1.252973,0.363191,-0.551733,-0.188170,-0.824310,0.293802,-1.085425,-0.482210,-0.531524,4.351412,-0.507826,0.815013,-0.490781,-0.271607,0.998910,-0.869888
913,1.536257,-0.516309,0.636973,-0.968441,1.812470,0.165420,-0.824310,2.356860,0.921298,-0.482210,-0.531524,-0.229810,-0.507826,0.815013,-0.490781,-0.271607,0.998910,-0.869888
914,0.370075,-0.516309,-0.141240,-2.131275,-0.551733,-0.856061,1.213136,0.293802,0.921298,-0.482210,-0.531524,-0.229810,-0.507826,0.815013,-0.490781,-0.271607,0.998910,-0.869888
915,0.370075,1.936826,-0.141240,-0.161960,-0.551733,1.461915,-0.824310,-0.831502,-1.085425,2.073784,-0.531524,-0.229810,1.969177,-1.226974,-0.490781,-0.271607,0.998910,-0.869888


In [52]:
previsoresdf.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02
mean,1.859654e-16,7.748558e-18,7.767929e-16,-1.869340e-16,4.649135e-17,-5.114048e-16,-1.046055e-16,7.748558e-17,-8.135986e-17,-2.324567e-17,1.549712e-17,7.748558e-18,3.099423e-17,-1.084798e-16,1.472226e-16,4.649135e-17,5.617704e-17,-3.099423e-17
std,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00
min,-2.704405e+00,-5.163086e-01,-2.920572e+00,-2.994023e+00,-5.517333e-01,-3.016886e+00,-8.243101e-01,-3.269662e+00,-1.085425e+00,-4.822104e-01,-5.315237e-01,-2.298105e-01,-5.078263e-01,-1.226974e+00,-4.907810e-01,-2.716072e-01,-1.001091e+00,-8.698879e-01
25%,-6.900904e-01,-5.163086e-01,-6.971063e-01,-5.745784e-01,-5.517333e-01,-6.596226e-01,-8.243101e-01,-8.315022e-01,-1.085425e+00,-4.822104e-01,-5.315237e-01,-2.298105e-01,-5.078263e-01,-1.226974e+00,-4.907810e-01,-2.716072e-01,-1.001091e+00,-8.698879e-01
50%,5.202558e-02,-5.163086e-01,-1.412398e-01,0.000000e+00,-5.517333e-01,4.755658e-02,-8.243101e-01,-2.688500e-01,9.212982e-01,-4.822104e-01,-5.315237e-01,-2.298105e-01,-5.078263e-01,8.150134e-01,-4.907810e-01,-2.716072e-01,9.989101e-01,-8.698879e-01
75%,6.881250e-01,-5.163086e-01,4.146267e-01,4.194568e-01,-5.517333e-01,7.547357e-01,1.213136e+00,5.751284e-01,9.212982e-01,-4.822104e-01,-5.315237e-01,-2.298105e-01,-5.078263e-01,8.150134e-01,-4.907810e-01,-2.716072e-01,9.989101e-01,1.149573e+00
max,2.490407e+00,1.936826e+00,3.749826e+00,6.721265e+00,1.812470e+00,2.561971e+00,1.213136e+00,4.982571e+00,9.212982e-01,2.073784e+00,1.881384e+00,4.351412e+00,1.969177e+00,8.150134e-01,2.037569e+00,3.681787e+00,9.989101e-01,1.149573e+00


### **Resumo:**
alvo = variável que se pretende atingir (tem ou não doença cardíaca).

previsores = conjunto de variáveis previsoras com as variáveis categóricas transformadas em numéricas manualmente, sem escalonar.

previsores_esc = conjunto de variáveis previsoras com as variáveis categóricas transformadas em numéricas, escalonada.

----

### **Base de treino e teste**

In [55]:
x_treino, x_teste,y_treino,y_teste = train_test_split(previsores_esc, alvo, test_size = 0.3, random_state = 10)

In [56]:
x_treino.shape,y_treino.shape

((641, 18), (641,))

In [57]:
x_teste.shape, y_teste.shape

((276, 18), (276,))

### **Criação do algoritmo**

In [71]:
#MLPClassifier?

### **Parâmetros MLPClassifier**
   
- hidden_layer_sizes (camadas escondidas): default (100,)   

  Quant.= (Ne+Ns)/2 = (11+1)/2 = 6 neurônios   
  Quant.=2/3.(Ne) + Ns = 2/3.11+1 = 8 neurônios
- activation: Função de ativação default='relu'
- solver: algoritmo matemático. Default='adam' (datasets grandes = acima de 1000 amostras). lbfgs é para datasets pequenos. sgd é com a descida do gradiente estocástico (recomendado testar).
- alpha:  parâmetro para o termo de regularização de ajuste de pesos. Aumento de alpha estimula pesos menores e diminuição de alpha estimula pesos maiores. Default=0.0001.
- batch_size: tamanho dos mini lotes. default=min(200, n_samples). Não usar com o solver lbfgs.
- learning_rate: taxa de aprendizagem. default='constant'. Três tipos:   
  1- 'constant':uma taxa de aprendizado constante dada pela taxa de aprendizagem inicial.   
  2- 'invscaling': diminui gradualmente por: taxa efetiva = taxa inicial / t^power_t  
  3- 'adaptive': a taxa é dividida por 5 cada vez que em duas épocas consecutivas não diminuir o erro.
- learning_rate_init: taxa de aprendizagem inicial. Default=0.001
- max_iter int: Número máximo de iterações. default = 200.  ('sgd', 'adam').
- max_fun: Número máximo de chamadas de função de perda. Para 'lbfgs'. Default: 15000
- shuffle: default = True
Usado apenas quando solver = 'sgd' ou 'adam'.
- random_state: default = None
- tol:Tolerância para a otimização.Default=0.0001
- momentum: otimização do algoritmo 'sgd'. Default: 0.9.
- n_iter_no_change: Número máximo de épocas que não atinge a tolerância de melhoria. default = 10. Apenas para solver = 'sgd' ou 'adam'
- verbose : Mostra o progresso. default=False.



In [109]:
redes = {
    'MLP': MLPClassifier(hidden_layer_sizes =(10,10),activation='identity', solver='lbfgs',max_fun=20000,
                         max_iter=800,tol=0.0001, random_state = 10,verbose = True),


}

class rede():

  def __init__(self,redes,x_treino,x_teste,y_treino,y_teste):
    self.x_treino = x_treino
    self.x_teste = x_teste
    self.y_treino = y_treino
    self.y_teste = y_teste
    self.redes = redes
    self.rede_score = []

  def treino(self):

    for name, redess in self.redes.items():

        rede = redess.fit(self.x_treino,self.y_treino)
        
        resultado = rede.score(self.x_treino,self.y_treino)

        resultado1 = rede.score(self.x_teste,self.y_teste)
        
        
        self.rede_score.append({'Modelos': name, 'Acurácia Treino':'{:.2f}%'.format(resultado*100),
                                     'Acurácia Teste': '{:.2f}%'.format(resultado1*100)})

    md = pd.DataFrame(self.rede_score)  
    return md

  def previsao(self,rede):

    previsoes = rede.predict(self.x_treino)  

    previsoes1 = rede.predict(self.x_teste)

    kfold = KFold(n_splits = 30, shuffle = True, random_state = 5)

    resul = cross_val_score(rede,previsores_esc,alvo,cv=kfold)  

    print(f'\nRelatório Treino')
      
    print(f"-----------------------------------------------------------------------")
    print(f'Acurácia: {round(accuracy_score(self.y_treino,previsoes)*100,2)}%') 
    print(f"-----------------------------------------------------------------------")

    print(f'Validação cruzada Treino: {round(resul.mean()*100,2)}%')

    print(f"-----------------------------------------------------------------------")  
    print(f'Matriz de cofusão:\n {confusion_matrix(self.y_treino,previsoes)}')
    print(f"-----------------------------------------------------------------------")  
    print(f'Classifição:\n {classification_report(self.y_treino,previsoes)}') 
    print(f"-----------------------------------------------------------------------")

    print(f'\nRelatório Teste')
      
    print(f"-----------------------------------------------------------------------")
      
    print(f'Acurácia: {round(accuracy_score(self.y_teste,previsoes1)*100,2)}%')
      
    print(f"-----------------------------------------------------------------------")

    print(f'Validação cruzada Teste: {round(resul.mean()*100,2)}%')

    print(f"-----------------------------------------------------------------------")
        
    print(f'Matriz de cofusão:\n {confusion_matrix(self.y_teste,previsoes1)}')

    print(f"-----------------------------------------------------------------------")  

    print(f'Classifição:\n {classification_report(self.y_teste,previsoes1)}')
      
    print(f"-----------------------------------------------------------------------")

Redes = rede(redes,x_treino,x_teste,y_treino,y_teste)
Redes.treino()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          311     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.27175D+00    |proj g|=  5.04855D-01

At iterate    1    f=  4.51388D-01    |proj g|=  1.41131D-01

At iterate    2    f=  3.72487D-01    |proj g|=  6.02821D-02

At iterate    3    f=  3.36927D-01    |proj g|=  2.37042D-02

At iterate    4    f=  3.31185D-01    |proj g|=  1.17947D-02

At iterate    5    f=  3.27327D-01    |proj g|=  8.39739D-03

At iterate    6    f=  3.25799D-01    |proj g|=  6.87062D-03

At iterate    7    f=  3.25521D-01    |proj g|=  5.38359D-03

At iterate    8    f=  3.25396D-01    |proj g|=  1.49578D-03

At iterate    9    f=  3.25366D-01    |proj g|=  5.96970D-04

At iterate   10    f=  3.25354D-01    |proj g|=  2.73488D-04

At iterate   11    f=  3.25351D-01    |proj g|=  1.09654D-04

At iterate   12    f=  3.25351D-01    |proj g|=  1.15821D-04

At iterate   13    f=  3.2

 This problem is unconstrained.


,Modelos,Acurácia Treino,Acurácia Teste
0,MLP,87.21%,85.14%


### Previsão da rede

In [113]:
Redes.previsao(redes['MLP'])

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          311     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.27583D+00    |proj g|=  5.21005D-01

At iterate    1    f=  4.67088D-01    |proj g|=  1.40309D-01

At iterate    2    f=  3.88974D-01    |proj g|=  6.23413D-02

At iterate    3    f=  3.47482D-01    |proj g|=  2.82699D-02

At iterate    4    f=  3.40811D-01    |proj g|=  1.24605D-02

At iterate    5    f=  3.37380D-01    |proj g|=  1.22579D-02

At iterate    6    f=  3.34792D-01    |proj g|=  6.54448D-03

At iterate    7    f=  3.34590D-01    |proj g|=  6.93526D-03

At iterate    8    f=  3.34325D-01    |proj g|=  1.27263D-03

At iterate    9    f=  3.34278D-01    |proj g|=  1.01434D-03

At iterate   10    f=  3.34245D-01    |proj g|=  4.89338D-04

At iterate   11    f=  3.34237D-01    |proj g|=  2.38221D-04

At iterate   12    f=  3.34236D-01    |proj g|=  2.07034D-04

At iterate   13    f=  3.3

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          311     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.28192D+00    |proj g|=  5.16716D-01

At iterate    1    f=  4.67782D-01    |proj g|=  1.44392D-01

At iterate    2    f=  3.85766D-01    |proj g|=  6.33021D-02

At iterate    3    f=  3.42142D-01    |proj g|=  2.97631D-02

At iterate    4    f=  3.35303D-01    |proj g|=  1.29877D-02

At iterate    5    f=  3.31915D-01    |proj g|=  1.21991D-02

At iterate    6    f=  3.29057D-01    |proj g|=  6.50254D-03

At iterate    7    f=  3.28841D-01    |proj g|=  7.04340D-03

At iterate    8    f=  3.28573D-01    |proj g|=  1.29187D-03

At iterate    9    f=  3.28526D-01    |proj g|=  1.03628D-03

At iterate   10    f=  3.28497D-01    |proj g|=  5.17589D-04

At iterate   11    f=  3.28489D-01    |proj g|=  2.45542D-04

At iterate   12    f=  3.28487D-01    |proj g|=  1.79272D-04

At iterate   13    f=  3.2

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.


...

 This problem is unconstrained.
 This problem is unconstrained.



At iterate    7    f=  3.27127D-01    |proj g|=  7.63095D-03

At iterate    8    f=  3.26823D-01    |proj g|=  1.61882D-03

At iterate    9    f=  3.26768D-01    |proj g|=  1.21524D-03

At iterate   10    f=  3.26724D-01    |proj g|=  5.82040D-04

At iterate   11    f=  3.26713D-01    |proj g|=  3.16711D-04

At iterate   12    f=  3.26712D-01    |proj g|=  2.95155D-04

At iterate   13    f=  3.26711D-01    |proj g|=  9.27028D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
  311     13     14      1     0     0   9.270D-05   3.267D-01
  F =  0.32671060121365814     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL  

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.



At iterate    4    f=  3.37069D-01    |proj g|=  1.18627D-02

At iterate    5    f=  3.33517D-01    |proj g|=  1.21886D-02

At iterate    6    f=  3.30986D-01    |proj g|=  7.28487D-03

At iterate    7    f=  3.30827D-01    |proj g|=  7.66793D-03

At iterate    8    f=  3.30511D-01    |proj g|=  1.39382D-03

At iterate    9    f=  3.30462D-01    |proj g|=  1.00513D-03

At iterate   10    f=  3.30418D-01    |proj g|=  4.51517D-04

At iterate   11    f=  3.30411D-01    |proj g|=  2.31919D-04

At iterate   12    f=  3.30410D-01    |proj g|=  2.47641D-04

At iterate   13    f=  3.30409D-01    |proj g|=  5.43868D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tn

 This problem is unconstrained.


### Validação cruzada

In [ ]:
R